In [1]:
from nano_graphrag import GraphRAG, QueryParam
from examples.using_deepseek_as_llm import (
    deepseepk_model_if_cache,   # LLM ya listo y con cache opcional
    local_st_embeddings,
    WORKING_DIR,
)
from IPython.display import Markdown, display

# === Inicializa GraphRAG (ya indexaste antes con el mismo WORKING_DIR) ===
rag = GraphRAG(
    working_dir=WORKING_DIR,
    best_model_func=deepseepk_model_if_cache,
    cheap_model_func=deepseepk_model_if_cache,
    embedding_func=local_st_embeddings,
    enable_naive_rag=True,
    enable_llm_cache=False,
)

def show_md(text: str):
    display(Markdown(f"```\n{text.strip()}\n```"))

# === Naive (solo embeddings) ===
async def ask_naive(q: str, max_tokens: int = 180):
    prompt = f"{q}\n\nResponde en máximo 5 líneas claras y concisas."
    # OJO: no se pasan kwargs aquí; GraphRAG no acepta temperature/max_tokens.
    return await rag.aquery(prompt, QueryParam(mode="naive"))

# === GraphRAG Global (comunidades) ===
async def ask_global(q: str, max_tokens: int = 180):
    prompt = f"{q}\n\nResponde en máximo 5 líneas claras y concisas."
    return await rag.aquery(prompt, QueryParam(mode="global"))

# === Puro LLM (sin retrieval) usando la función del example ===
async def ask_llm_only(q: str, temperature: float = 0.7, max_tokens: int = 200):
    prompt = f"{q}\n\nResponde en máximo 5 líneas claras y concisas."
    # Esta función ya llama a DeepSeek con AsyncOpenAI y maneja cache si lo activas
    return await deepseepk_model_if_cache(
        prompt,
        temperature=temperature,
        max_tokens=max_tokens,
    )



/opt/miniconda3/envs/graphrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:nano-graphrag:Loading tokenizer: type='tiktoken', name='gpt-4o'
INFO:nano-graphrag:Load KV full_docs with 1 data
INFO:nano-graphrag:Load KV text_chunks with 42 data
INFO:nano-graphrag:Load KV community_reports with 98 data
INFO:nano-graphrag:Loaded graph from ./nano_graphrag_cache_deepseek_TEST/graph_chunk_entity_relation.graphml with 569 nodes, 601 edges
INFO:nano-vectordb:Load (556, 384) data
INFO:nano-vectordb:Init {'embedding_dim': 384, 'metric': 'cosine', 'storage_file': './nano_graphrag_cache_deepseek_TEST/vdb_entities.json'} 556 data
INFO:na

In [3]:
# === Ejemplo de comparación ===
question = "Donde vive Scrooge?"

ans_naive  = await ask_naive(question)
ans_global = await ask_global(question)
# ans_llm    = await ask_llm_only(question, temperature=0.5, max_tokens=180)

print("Pregunta:");   show_md(question)
print("NAIVE RAG:\n");   show_md(ans_naive)
print("\nGRAPH RAG (GLOBAL):\n"); show_md(ans_global)
# print("\nPURO LLM:\n");    show_md(ans_llm)

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.76it/s]
INFO:nano-graphrag:Truncate 20 to 9 chunks
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:nano-graphrag:Revtrieved 96 communities
INFO:nano-graphrag:Grouping to 5 groups for global search
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:nano-graphrag:JSON data successfully extracted.
INFO:nano-graphrag:JSON data successfully extracted.
INFO:nano-graphrag:JSON data successfully extracted.
INFO:nano-graphrag:JSON data successfully extracted.
INFO:nano-graphrag:JSON data successfully extra

Pregunta:


```
Donde vive Scrooge?
```

NAIVE RAG:



```
Según la información proporcionada, Scrooge vive en unas habitaciones que anteriormente pertenecieron a su socio fallecido, Jacob Marley. Se describe como un conjunto de habitaciones lúgubres en un edificio sombrío, situado en un patio. El texto especifica que es un lugar oscuro y que nadie más vivía en el edificio excepto Scrooge, ya que las demás habitaciones se alquilaban como oficinas.
```


GRAPH RAG (GLOBAL):



```
Scrooge vive en Londres, específicamente en una zona conocida como The City. Su residencia es una casa lúgubre y oscura que anteriormente perteneció a su difunto socio, Marley. Esta vivienda se encuentra en un edificio envuelto en niebla y escarcha, lo que refleja su carácter sombrío.
```